# Coding Assistant

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import re
import sys
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
from langchain.tools import StructuredTool
from typing import List
from enum import Enum, IntEnum

model =AzureChatOpenAI(deployment_name="gpt-4-32k", temperature=0)
application = "command line fibonacci calculator"
technology = "python"

In [8]:
# Create a featurelist for the tool we want to create
class Feature(BaseModel):
    feature: str = Field(description="The feature we want to create")
    description: str = Field(description="The description of the feature we want to create")

class Featurelist(BaseModel):
    features: List[Feature] = Field(description="A list of features we want to create")


feature_parser = PydanticOutputParser(pydantic_object=Featurelist)

# Create a prompt template for the tool we want to create
feature_prompt = PromptTemplate(
    template="I want to create a {application} using {technology}. Please give a short outline of which features should be implemented. .\n{format_instructions}\n",
    input_variables=["application", "technology"],
    partial_variables={"format_instructions": feature_parser.get_format_instructions()},
)

feature_chain = feature_prompt | model | feature_parser

def get_features(application: str, technology: str) -> Featurelist:
    """Provide a featurelist for a given application and technology"""
    features = feature_chain.invoke({"application": application, 
                                    "technology": technology})
    return features

features = StructuredTool.from_function(get_features)

In [10]:
# create a directory structure for the tool we want to create
class File(BaseModel):
    path: str = Field(description="The path of the file")
    responsibility: str = Field(description="The responsibility of the file, what it should contain and what the responsibility")
    dependencies: List[str] = Field(description="The other files this file depends on")

class Files(BaseModel):
    files: List[File] = Field(description="A list of files we want to create")

file_parser = PydanticOutputParser(pydantic_object=Files) 


directory_prompt = PromptTemplate(
    template="I want to create a {application} using {technology} with these features: {features}. List all the files that are needed and use this list to provide a file structure for this application. Describe what each file is responsible for and include supporting files like requirements.txt, README.md and setup files when needed: .\n{format_instructions}\n",
    input_variables=["application", "technology", "features"],
    partial_variables={"format_instructions": file_parser.get_format_instructions()},
)

def get_directory_structure(application: str, technology: str, features: Featurelist) -> Files:
    """Provide a file structure for a given application, technology and Features"""
    feature_list = ', '.join([feature.feature for feature in features.features])
    directory_chain = directory_prompt | model | file_parser
    directory_entries = directory_chain.invoke({"application": application, 
                                                "technology": technology,
                                                "features": feature_list})
    return directory_entries

directory = StructuredTool.from_function(get_directory_structure)

In [57]:
# create all the files for the tool we want to create
base_dir = os.path.join(os.getcwd(), application + "/")
features = get_features(application, technology)
feature_list = ', '.join([feature.feature for feature in features.features])

directory_entries = get_directory_structure(application, technology, features)

files_to_create = directory_entries.files.copy()    

code_prompt = PromptTemplate(
    template="I want to create a {application} using {technology} with these features: {features}. Please provide the contents for the file {file}, which is responsible for {responsibility}.  Please include inline documentation where needed. \n",
    input_variables=["application", "technology", "file", "responsibility", "features"],
)

code_chain = code_prompt | model
def get_code(application: str, technology: str, file: File, features: str) -> str:
    print(f"Getting code for file {file.path}")
    code = code_chain.invoke({"application": application,
                              "technology": technology,
                              "file": file.path,
                              "responsibility": file.responsibility,
                              "features": features})
    pattern = "```(.*?)(.+)```"    
    result = re.search(pattern, code.content, re.DOTALL).group(2)

    print(f"Code for file {file.path} is {result}")    
    sys.exit()
    return result


def create_file(file: File):
    print(f"Creating file {file.path}")
    for dependency in file.dependencies:
        print(f"Checking if dependency {dependency} exists")        
        dependency_file = next((f for f in directory_entries.files if f.path == dependency), None)
        print(f"Creating dependency file {dependency_file.path} first")
        create_file(dependency_file)
    if not os.path.dirname(base_dir + file.path) == "":
        os.makedirs(os.path.dirname(base_dir + file.path), exist_ok=True)
    with open(base_dir + file.path, "w") as f:
        code = get_code(application, technology, file, feature_list)
        f.write(code)
        file_to_remove = next((f for f in files_to_create if f.path == file.path), None)
        if file_to_remove:
            files_to_create.remove(file_to_remove)

while len(files_to_create) > 0:
    create_file(files_to_create[0])

Creating file README.md
Getting code for file README.md
Code for file README.md is <re.Match object; span=(2105, 2179), match='```json\n{\n    "0": 0,\n    "1": 1,\n    "2": 1,>


SystemExit: 

/home/johann/anaconda3/envs/bahn/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
